****"Gibbs Sampler algorithm to search for motifs in the 'Dormancy Survival regulators (DosR)' upstream genes dataset ('https://bioinformaticsalgorithms.com/data/challengedatasets/DosR.txt') of Mycobacterium tuberculosis, with k-mer length equal to 15 for 20 different starts, each time with N (number of runs) = 100"****



In [ ]:
"Download the sequence of 10 genes of DosR dataset."

import wget

DosR = wget.download('https://bioinformaticsalgorithms.com/data/challengedatasets/DosR.txt')

In [ ]:
with open('DosR.txt','r') as DosR_seq:
    
    DosR = []
    for i in DosR_seq:
        DosR.append(i.rstrip())
DosR

In [ ]:

# import the random package
import random

def GibbsSampler(Dna, k, t, N):
    BestMotifs = [] # output variable
    Motifs = RandomMotifs(Dna, k, t)
    #print(Motifs)
    BestMotifs = Motifs
    for j in range(N):
        i = random.randint(0,t-1)
        new_Motif = []
        for k1 in range(t):
            if k1!=i:
                new_Motif.append(Motifs[k1])
        profile = ProfileWithPseudocounts(new_Motif)
        motif_i = ProfileGeneratedString(Dna[i], profile, k)
        Motifs[i] = motif_i
        if Score(Motifs)<Score(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs

# place all subroutines needed for GibbsSampler below this line
def RandomMotifs(Dna, k, t):
    r_motifs = []
    r = len(Dna[0])
    for i in range(t):
        x = random.randint(0,r-k)
        r_motifs.append(Dna[i][x:x+k])
    return r_motifs

def CountWithPseudocounts(Motifs):
    count = {} # initializing the count dictionary
    t = len(Motifs)
    k = len(Motifs[0])
    #print(Motifs)
    for symbol in "ACGT":
        count[symbol] = []
        for j in range(k):
            count[symbol].append(0)
    for i in range(t):
        for j in range(k):
            
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    for i in count.items():
        j = i[1]
        for x in range(len(j)):
            j[x] = j[x] + 1
    return count

def ProfileWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    profile = {}
    profile=CountWithPseudocounts(Motifs)
    for i in "ATCG":
        for j in range(k):
            profile[i][j] = profile[i][j]/ (t + 4)
    return profile

def ProfileGeneratedString(Text, profile, k):
    n = len(Text)
    probabilities = {} 
    for i in range(0,n-k+1):
        probabilities[Text[i:i+k]] = Pr(Text[i:i+k], profile)
        #print(probabilities)
    probabilities = Normalize(probabilities)
    return WeightedDie(probabilities)

def Normalize(Probabilities):
    result = {}
    sum = 0
    for m in Probabilities:
        sum += Probabilities[m]
    for n in Probabilities:
        result[n]= Probabilities[n]/sum
    return result  

def WeightedDie(Probabilities):
    kmer = ''
    p = random.uniform(0,1)
    x = 0
    for k,v in Probabilities.items():
        if p>=x and p<=v+x:
            kmer = kmer + k
            return kmer
        else:
            x += v

def Pr(Text, Profile):
    p = 1
    for i in range(len(Text)):
        p = p * Profile[Text[i]][i]
    return p

def Score(Motifs):
    k = len(Motifs[0])
    count = CountWithPseudocounts(Motifs)
    consensus = Consensus(Motifs)
    sum=0
    row=0
    for i in consensus:
        sum1=0
        for j in count:
            if j!= i:
                sum1 += count[j][row]
        sum += sum1
        row +=1
    return sum

def Consensus(Motifs):
    count = CountWithPseudocounts(Motifs)
    consensus = ""
    k = len(Motifs[0])
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus

# Copy DosR dataset below.
Dna = DosR

# set t equal to the number of strings in Dna, k equal to 15, and N equal to 100
t = len(Dna)
k = 15
N = 100

# Call GibbsSampler(Dna, k, t, N) 20 times and store the best output in a variable called BestMotifs
Motifs = GibbsSampler(Dna, k, t, N)
BestMotifs = Motifs
for i in range(20):
    if Score(Motifs)<Score(BestMotifs):
            BestMotifs = Motifs
    
# Print the BestMotifs variable
print(BestMotifs)
# Print Score(BestMotifs)
print(Score(BestMotifs))